# Implementación preliminar del código

### Descargar paquetes necesarios

In [49]:
#!pip install catboost
#!pip install sweetviz
#!pip install optuna
#!pip install lightgbm
#!pip install scikit-learn
#!pip install xgboost
#!pip install pyreadstat

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/565.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/565.3 kB ? eta -:--:--
     -------------------------------------- 565.3/565.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for pyreadstat: filename=pyreadstat-1.2.9-cp38-cp38-win_amd64.whl size=319963 sha256=57481a12f9b06739deaee8ee58afc526a116a36b435b68ce32f16f777c63cea6
  Stored in directory: c:\users\josé carlos\appdata\local\pip\cache\wheels\34\62\02\7d2524ef9e4a75c2ae00c41eebb26434b133adb534db354842
Successfully built pyreadstat


In [1]:
# --- Librerías estándar ---
import os  # Rutas de los archivos

# --- Librerías científicas y de análisis ---
import matplotlib.pyplot as plt # Gráficos
import numpy as np  # Funciones matemáticas
import pandas as pd  # Manejo de dataframes
import sweetviz as sviz # EDA

# --- Modelos de clasificación ---
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import (AdaBoostClassifier, BaggingClassifier, 
                              ExtraTreesClassifier, GradientBoostingClassifier, 
                              RandomForestClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

# --- Metricas y evaluación --- 
from sklearn.metrics import (
    accuracy_score, precision_score, f1_score, recall_score
)
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline

# --- Optimizador ---
import optuna

# --- Elaboración propia ---
from ProbadorHipotesis import ProbadorHipotesis
from ModeladorDatos import ModeladorDatos
from ImportadorDatos import ImportadorDatos
from OptimizadorExtraTreesOptuna import OptimizadorExtraTreesOptuna

C:\Users\José Carlos\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Lectura de datos

Note que la limpieza y transformación de los datos se efectuó en el script "main_enaho.py"

In [2]:
ruta_actual = os.getcwd()
ruta_datos = os.path.join(ruta_actual, '../..', 'data/base.csv') 
ruta_normalizada = os.path.normpath(ruta_datos) 
base = pd.read_csv(ruta_normalizada)

In [3]:
base.head()

,ZONA,REGION,V1,V2A,V3,V4,V5,V6,V6A,V7A,...,V19,V21,R4A,TamViv,A4,A5,A6,CondMig,NivInst,A22A
0,Urbana,Central,En fila o contigua,Alquiler o cesión,Block o ladrillo,Lámina de metal o zinc,Sí,"Mosaico, cerámica, terrazo",No,Bueno,...,SÍ,0,No,2.0,Hombre,45 a 59,Separado/Divorciado/Viudo,No migrante,Secundaria completa,Sí
1,Urbana,Central,En fila o contigua,Alquiler o cesión,Block o ladrillo,Lámina de metal o zinc,Sí,"Mosaico, cerámica, terrazo",No,Bueno,...,SÍ,0,No,2.0,Mujer,13 a 17,Soltero(a),Migrante interno,Primaria completa,No
2,Urbana,Central,En fila o contigua,Propiedad,Otro,Lámina de metal o zinc,Sí,"Mosaico, cerámica, terrazo",No,Bueno,...,SÍ,0,No,3.0,Mujer,45 a 59,Casado(a),Migrante externo,Primaria completa,No
3,Urbana,Central,En fila o contigua,Propiedad,Otro,Lámina de metal o zinc,Sí,"Mosaico, cerámica, terrazo",No,Bueno,...,SÍ,0,No,3.0,Hombre,60 a 74,Casado(a),Migrante externo,Primaria completa,Sí
4,Urbana,Central,En fila o contigua,Propiedad,Otro,Lámina de metal o zinc,Sí,"Mosaico, cerámica, terrazo",No,Bueno,...,SÍ,0,No,3.0,Mujer,25 a 34,Soltero(a),No migrante,Superior,Sí


## Se evaluan los modelos

In [12]:
models = {
    "LogisticRegression": LogisticRegression(),
    "DecisionTree": DecisionTreeClassifier(),
    "SVM": SVC(probability=True),
    "RandomForest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(),
    "KNN": KNeighborsClassifier(),
    "NeuralNetwork": MLPClassifier(),
    "LDA": LinearDiscriminantAnalysis(),
    "CatBoost": CatBoostClassifier(),
    "LightGBM": LGBMClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "GradientBoosting": GradientBoostingClassifier(),
    "NaiveBayes": GaussianNB(),
    "QDA": QuadraticDiscriminantAnalysis(),
    "Bagging": BaggingClassifier(base_estimator=DecisionTreeClassifier()),
    "ExtraTrees": ExtraTreesClassifier()
}
modelador = ModeladorDatos(df = base, var_objetivo = 'V21', modelos = models)

#### evaluacion de los modelos con el modelador

In [13]:
results_df = modelador.evaluar_modelos()

Training LogisticRegression...


C:\Users\José Carlos\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Training DecisionTree...
Training SVM...
Training RandomForest...
Training XGBoost...
Training KNN...
Training NeuralNetwork...


C:\Users\José Carlos\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training LDA...
Training CatBoost...
Learning rate set to 0.036085
0:	learn: 0.6656510	total: 169ms	remaining: 2m 48s
1:	learn: 0.6406955	total: 201ms	remaining: 1m 40s
2:	learn: 0.6167951	total: 238ms	remaining: 1m 19s
3:	learn: 0.5962398	total: 272ms	remaining: 1m 7s
4:	learn: 0.5769672	total: 293ms	remaining: 58.3s
5:	learn: 0.5593462	total: 311ms	remaining: 51.6s
6:	learn: 0.5440616	total: 330ms	remaining: 46.8s
7:	learn: 0.5288220	total: 349ms	remaining: 43.3s
8:	learn: 0.5159265	total: 368ms	remaining: 40.5s
9:	learn: 0.5071103	total: 388ms	remaining: 38.4s
10:	learn: 0.4965033	total: 407ms	remaining: 36.6s
11:	learn: 0.4857068	total: 427ms	remaining: 35.1s
12:	learn: 0.4778379	total: 445ms	remaining: 33.8s
13:	learn: 0.4697625	total: 463ms	remaining: 32.6s
14:	learn: 0.4619196	total: 482ms	remaining: 31.6s
15:	learn: 0.4562249	total: 502ms	remaining: 30.9s
16:	learn: 0.4514157	total: 519ms	remaining: 30s
17:	learn: 0.4463129	total: 536ms	remaining: 29.3s
18:	learn: 0.4409676	tot

C:\Users\José Carlos\AppData\Roaming\Python\Python38\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Training Bagging...


C:\Users\José Carlos\AppData\Roaming\Python\Python38\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Training ExtraTrees...


In [14]:
# Guardar en Excel
ruta_resultado_modelos = os.path.join(ruta_actual, '../..', 'res/resultados_modelos.xlsx')
ruta_resultado_modelos_norm = os.path.normpath(ruta_resultado_modelos)
results_df.drop(columns=["Predictions"]).to_excel(ruta_resultado_modelos_norm, index=False)
print(results_df)

                 Model  Accuracy   ROC AUC      Gini  Precision    Recall  \
0   LogisticRegression  0.839445  0.851219  0.702438   0.666337  0.412377   
1         DecisionTree  0.914148  0.866825  0.733651   0.787860  0.787377   
2                  SVM  0.865337  0.889817  0.779634   0.805151  0.440564   
3         RandomForest  0.943880  0.980406  0.960812   0.955212  0.757966   
4              XGBoost  0.900768  0.943868  0.887736   0.822844  0.648897   
5                  KNN  0.886397  0.913744  0.827487   0.765011  0.632353   
6        NeuralNetwork  0.911794  0.944811  0.889622   0.771226  0.801471   
7                  LDA  0.834490  0.836813  0.673626   0.649495  0.393995   
8             CatBoost  0.895937  0.936683  0.873365   0.836735  0.602941   
9             LightGBM  0.878097  0.915967  0.831935   0.780763  0.552083   
10            AdaBoost  0.837587  0.845573  0.691146   0.667712  0.391544   
11    GradientBoosting  0.846259  0.865013  0.730025   0.714599  0.398897   

Se considera que el modelo de ExtraTrees es el más adecuado dado a su alto score tanto en precisión como en recall, por lo que se va a utilizar para la optimización de parámetros

### Se crea la funcion a optimizar

In [15]:
optimizador = OptimizadorExtraTreesOptuna(base, "V21", 75)

In [16]:
optimizador.optimizar()

[I 2025-06-07 18:17:47,932] A new study created in memory with name: no-name-f919e650-017e-4fea-bffd-1a409242f103
[I 2025-06-07 18:18:22,644] Trial 0 finished with value: 0.749813836956462 and parameters: {'n_estimators': 343, 'max_depth': 21, 'min_samples_split': 30, 'min_samples_leaf': 9, 'max_features': 0.9473456101130713, 'max_leaf_nodes': 242}. Best is trial 0 with value: 0.749813836956462.
[I 2025-06-07 18:19:11,780] Trial 1 finished with value: 0.7094144866564224 and parameters: {'n_estimators': 496, 'max_depth': 8, 'min_samples_split': 22, 'min_samples_leaf': 3, 'max_features': 0.9201705436391319, 'max_leaf_nodes': 62}. Best is trial 0 with value: 0.749813836956462.
[I 2025-06-07 18:19:28,351] Trial 2 finished with value: 0.6814073286682464 and parameters: {'n_estimators': 396, 'max_depth': 6, 'min_samples_split': 23, 'min_samples_leaf': 14, 'max_features': 0.33461795924236976, 'max_leaf_nodes': 129}. Best is trial 0 with value: 0.749813836956462.
[I 2025-06-07 18:19:53,192] Tr

Mejores hiperparámetros encontrados:
{'n_estimators': 225, 'max_depth': 20, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': 0.4907712448485949, 'max_leaf_nodes': 300}
Mejor F1 (macro): 0.8037393995147449


"Parámetros: {'n_estimators': 225, 'max_depth': 20, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': 0.4907712448485949, 'max_leaf_nodes': 300}\nScore: 0.8037393995147449"

In [17]:
modelo_optimizado = optimizador.entrenar_modelo_final()
modelo_optimizado

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['V18A1']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['ZONA', 'REGION', 'V1',
                                                   'V2A', 'V3', 'V4', 'V5',
                                                   'V6', 'V6A', 'V7A', 'V7B',
                                                   'V7C', 'EFI', 'V8', 'V11',
                                                   'V12', 'V13A', 'V14A1',
                                                   'V15', 'V16', 'V17A',
                                                   'V17B1', 'V17B2', 'V17B3',
                                                   'V17B4', 'V17B5', 'V18C',
                                                   'V18D', 'V18E', 'V18F', ...])],
                                   verbose_feature_names_out=False)),
                ('classifier',
                 ExtraTreesClassifier(max_depth=20,
                                      max_features=0.4907712448485949,
                                      max_leaf_nodes=300, min_samples_leaf=3,
                                      n_estimators=225, n_jobs=-1,
                                      random_state=42))])

#### Se calculan predicciones con distintas probabilidades y se generan los resultados

In [18]:
optimizador.evaluar_modelo_final()

,Accuracy,Precisión,Recall,F1
0,0.202180,0.202180,1.000000,0.336356
1,0.716551,0.409192,0.905637,0.563692
2,0.845391,0.583117,0.825368,0.683409
3,0.884167,0.701096,0.744485,0.722140
4,0.895441,0.786754,0.662377,0.719228
5,0.893335,0.845121,0.578431,0.686795
6,0.887884,0.899012,0.501838,0.644121
7,0.872151,0.945104,0.390319,0.552472
8,0.840932,0.980663,0.217525,0.356068
9,0.801536,1.000000,0.018382,0.036101


A partir de estos resultados, se observa que el desempeño global del modelo (según la métrica F1) es óptimo cuando se escoge un umbral de probabilidad alrededor de 30%, lo que representa el mejor balance entre precisión y sensibilidad para predecir la tenencia de bono de vivienda.

#### Importancia de las variables de la ENAHO

In [19]:
importancias = modelador.importancia_variables_modelos("ExtraTrees", modelo_optimizado)

Evaluando importancia para ExtraTrees...


In [20]:
importancias["ExtraTrees"].iloc[1:10]

,Variable,Importancia
13,V2A_Propiedad,0.086519
12,V2A_Alquiler o cesión,0.043536
15,V3_Block o ladrillo,0.039135
61,V15_De la CNFL,0.035600
2,ZONA_Urbana,0.023356
1,ZONA_Rural,0.022871
4,REGION_Central,0.022058
57,V14A1_1.0,0.015517
0,V18A1,0.015088
